# 🔬 Análise Interativa de Dados Sísmicos

Esta aplicação permite analisar dados sísmicos de forma interativa. Carregue seu arquivo CSV e obtenha análises automáticas de energia e detecção de ruídos.

## 📋 Como usar:
1. **Carregue seu arquivo**: Use o botão abaixo para selecionar um arquivo CSV
2. **Aguarde o processamento**: A análise será executada automaticamente
3. **Visualize os resultados**: Gráficos e estatísticas serão exibidos
4. **Baixe os resultados**: Arquivo processado estará disponível para download

In [ ]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import io
import base64
from datetime import datetime
import warnings

# Configurações
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("✅ Bibliotecas carregadas com sucesso!")

In [ ]:
# Widget para upload de arquivo
file_upload = widgets.FileUpload(
    accept='.csv',
    multiple=False,
    description='Carregar CSV',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Botão para processar
process_button = widgets.Button(
    description='🔄 Processar Análise',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='40px')
)

# Área de status
status_output = widgets.Output()

# Área de resultados
results_output = widgets.Output()

# Interface
display(widgets.VBox([
    widgets.HTML("<h3>📁 Carregar Arquivo de Dados Sísmicos</h3>"),
    widgets.HTML("<p>Selecione um arquivo CSV com dados de eventos sísmicos (formato esperado: event_id, plane, sensor, x_amplitude, y_amplitude, z_amplitude)</p>"),
    file_upload,
    process_button,
    status_output,
    results_output
]))

In [ ]:
def validate_csv_format(df):
    """Valida se o CSV está no formato correto"""
    required_columns = ['event_id', 'plane', 'sensor', 'x_amplitude', 'y_amplitude', 'z_amplitude']
    
    # Verificar colunas obrigatórias
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        return False, f"Colunas faltando: {missing_columns}"
    
    # Verificar se amplitudes são numéricas
    amplitude_cols = ['x_amplitude', 'y_amplitude', 'z_amplitude']
    for col in amplitude_cols:
        if not pd.api.types.is_numeric_dtype(df[col]):
            try:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            except:
                return False, f"Coluna {col} não pode ser convertida para numérico"
    
    # Verificar valores nulos
    null_counts = df[amplitude_cols].isnull().sum()
    if null_counts.sum() > 0:
        return False, f"Valores nulos encontrados: {null_counts.to_dict()}"
    
    return True, "Formato válido"

def calculate_energy_analysis(df):
    """Calcula análise de energia e detecta ruídos"""
    
    # Calcular energia total
    df['total_energy'] = np.sqrt(
        df['x_amplitude']**2 + 
        df['y_amplitude']**2 + 
        df['z_amplitude']**2
    )
    
    # Adicionar ranking e percentil
    df['energy_rank'] = df['total_energy'].rank(ascending=False)
    df['energy_percentile'] = df['total_energy'].rank(pct=True)
    
    # Estatísticas por evento
    event_stats = df.groupby('event_id')['total_energy'].agg(['mean', 'std', 'count']).add_prefix('event_')
    df = df.merge(event_stats, left_on='event_id', right_index=True)
    
    # Detectar ruídos
    high_energy_threshold = df['total_energy'].quantile(0.95)
    df['high_energy_flag'] = df['total_energy'] > high_energy_threshold
    
    high_variance_threshold = event_stats['event_std'].quantile(0.9)
    df['potential_noise_event'] = df['event_std'] > high_variance_threshold
    
    # Classificar energia por evento
    energy_means = df.groupby('event_id')['total_energy'].mean()
    low_energy_threshold = energy_means.quantile(0.33)
    high_energy_event_threshold = energy_means.quantile(0.67)
    
    def classify_energy(event_id):
        mean_energy = energy_means[event_id]
        if mean_energy <= low_energy_threshold:
            return 'Baixa'
        elif mean_energy <= high_energy_event_threshold:
            return 'Média'
        else:
            return 'Alta'
    
    df['event_energy_class'] = df['event_id'].apply(classify_energy)
    
    return df

def create_analysis_plots(df):
    """Cria gráficos de análise"""
    
    # Configurar figura
    fig = plt.figure(figsize=(16, 12))
    
    # Preparar dados para gráficos
    event_stats = df.groupby('event_id').agg({
        'total_energy': ['mean', 'std', 'count']
    }).round(8)
    event_stats.columns = ['mean', 'std', 'count']
    event_stats = event_stats.dropna()
    
    high_variance_threshold = event_stats['std'].quantile(0.9)
    noise_events = event_stats[event_stats['std'] > high_variance_threshold].index
    clean_events = event_stats[event_stats['std'] <= high_variance_threshold].index
    
    # 1. Comparação de quantidade: Ruído vs Limpo
    plt.subplot(2, 3, 1)
    noise_count = len(noise_events)
    clean_count = len(clean_events)
    categories = ['Eventos\nLimpos', 'Eventos\ncom Ruído']
    counts = [clean_count, noise_count]
    colors = ['lightgreen', 'salmon']
    
    bars = plt.bar(categories, counts, color=colors, alpha=0.7, edgecolor='black')
    plt.ylabel('Quantidade de Eventos')
    plt.title('Eventos: Ruído vs Limpos')
    plt.grid(True, alpha=0.3)
    
    # Adicionar valores nas barras
    for i, v in enumerate(counts):
        plt.text(i, v + max(counts)*0.01, str(v), ha='center', va='bottom', fontweight='bold')
    
    # 2. Distribuição de energia: Ruído vs Limpo
    plt.subplot(2, 3, 2)
    noise_energies = df[df['event_id'].isin(noise_events)]['total_energy']
    clean_energies = df[df['event_id'].isin(clean_events)]['total_energy']
    
    plt.hist([clean_energies, noise_energies], bins=30, alpha=0.7, 
             color=['lightgreen', 'salmon'], label=['Eventos Limpos', 'Eventos com Ruído'], 
             edgecolor='black')
    plt.xlabel('Energia Total')
    plt.ylabel('Frequência')
    plt.title('Distribuição de Energia')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.yscale('log')
    
    # 3. Energia por Classe
    plt.subplot(2, 3, 3)
    energy_class_counts = df['event_energy_class'].value_counts()
    colors_class = ['lightblue', 'orange', 'lightcoral']
    plt.pie(energy_class_counts.values, labels=energy_class_counts.index, 
            autopct='%1.1f%%', colors=colors_class, startangle=90)
    plt.title('Distribuição por Classe de Energia')
    
    # 4. Scatter plot: Energia vs Event ID
    plt.subplot(2, 3, 4)
    plt.scatter(df[df['event_id'].isin(clean_events)]['event_id'], 
              df[df['event_id'].isin(clean_events)]['total_energy'], 
              alpha=0.6, s=10, color='lightgreen', label='Eventos Limpos')
    plt.scatter(df[df['event_id'].isin(noise_events)]['event_id'], 
              df[df['event_id'].isin(noise_events)]['total_energy'], 
              alpha=0.8, s=15, color='red', label='Eventos com Ruído')
    plt.xlabel('Event ID')
    plt.ylabel('Energia Total')
    plt.title('Energia por Event ID')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 5. Box plot comparativo
    plt.subplot(2, 3, 5)
    data_for_boxplot = [clean_energies, noise_energies]
    box_plot = plt.boxplot(data_for_boxplot, labels=['Limpos', 'Ruído'], 
                          patch_artist=True)
    box_plot['boxes'][0].set_facecolor('lightgreen')
    box_plot['boxes'][1].set_facecolor('salmon')
    plt.ylabel('Energia Total')
    plt.title('Comparação Estatística')
    plt.grid(True, alpha=0.3)
    plt.yscale('log')
    
    # 6. Matriz de correlação
    plt.subplot(2, 3, 6)
    correlation_data = df[['x_amplitude', 'y_amplitude', 'z_amplitude', 'total_energy']]
    correlation_matrix = correlation_data.corr()
    
    im = plt.imshow(correlation_matrix, cmap='coolwarm', aspect='auto')
    plt.colorbar(im, shrink=0.8)
    plt.xticks(range(len(correlation_matrix.columns)), correlation_matrix.columns, rotation=45)
    plt.yticks(range(len(correlation_matrix.columns)), correlation_matrix.columns)
    plt.title('Correlação entre Componentes')
    
    # Adicionar valores na matriz
    for i in range(len(correlation_matrix.columns)):
        for j in range(len(correlation_matrix.columns)):
            plt.text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}', 
                    ha='center', va='center', color='black', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    return fig

def generate_summary_stats(df):
    """Gera estatísticas resumidas"""
    
    total_records = len(df)
    unique_events = df['event_id'].nunique()
    unique_sensors = df['sensor'].nunique()
    unique_planes = df['plane'].nunique()
    
    high_energy_count = df['high_energy_flag'].sum()
    noise_events_count = len(df[df['potential_noise_event'] == True]['event_id'].unique())
    
    clean_events_count = unique_events - noise_events_count
    
    energy_stats = df['total_energy']
    
    stats_html = f"""
    <div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px; margin: 10px 0;">
        <h2>📊 Resumo da Análise</h2>
        <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 20px;">
            <div>
                <h3>📈 Estatísticas Gerais</h3>
                <ul>
                    <li><strong>Total de registros:</strong> {total_records:,}</li>
                    <li><strong>Eventos únicos:</strong> {unique_events:,}</li>
                    <li><strong>Sensores únicos:</strong> {unique_sensors}</li>
                    <li><strong>Planos únicos:</strong> {unique_planes}</li>
                </ul>
            </div>
            <div>
                <h3>⚡ Estatísticas de Energia</h3>
                <ul>
                    <li><strong>Energia média:</strong> {energy_stats.mean():.6e}</li>
                    <li><strong>Energia mediana:</strong> {energy_stats.median():.6e}</li>
                    <li><strong>Desvio padrão:</strong> {energy_stats.std():.6e}</li>
                    <li><strong>Coef. variação:</strong> {(energy_stats.std() / energy_stats.mean()):.3f}</li>
                </ul>
            </div>
        </div>
        <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 20px; margin-top: 20px;">
            <div>
                <h3>🔍 Detecção de Ruídos</h3>
                <ul>
                    <li><strong>Eventos limpos:</strong> {clean_events_count} ({clean_events_count/unique_events*100:.1f}%)</li>
                    <li><strong>Eventos com ruído:</strong> {noise_events_count} ({noise_events_count/unique_events*100:.1f}%)</li>
                    <li><strong>Registros alta energia:</strong> {high_energy_count}</li>
                </ul>
            </div>
            <div>
                <h3>📋 Classes de Energia</h3>
                <ul>
    """
    
    # Adicionar distribuição por classe
    energy_class_dist = df['event_energy_class'].value_counts()
    for class_name, count in energy_class_dist.items():
        percentage = count / len(df) * 100
        stats_html += f"<li><strong>{class_name}:</strong> {count} ({percentage:.1f}%)</li>"
    
    stats_html += """
                </ul>
            </div>
        </div>
    </div>
    """
    
    return stats_html

def create_download_link(df, filename):
    """Cria link para download do CSV processado"""
    csv_string = df.to_csv(index=False)
    b64 = base64.b64encode(csv_string.encode()).decode()
    
    download_html = f"""
    <div style="background-color: #e8f5e8; padding: 15px; border-radius: 8px; margin: 10px 0;">
        <h3>💾 Download dos Resultados</h3>
        <p>Clique no link abaixo para baixar o arquivo CSV com todas as análises:</p>
        <a href="data:text/csv;base64,{b64}" download="{filename}" 
           style="background-color: #4CAF50; color: white; padding: 10px 20px; 
                  text-decoration: none; border-radius: 5px; font-weight: bold;">
           📥 Baixar {filename}
        </a>
        <p style="margin-top: 10px; font-size: 0.9em; color: #666;">
            Arquivo contém {len(df)} registros com energia calculada, classificação de ruídos e estatísticas por evento.
        </p>
    </div>
    """
    
    return download_html

print("✅ Funções de análise carregadas!")

In [ ]:
def process_analysis(button):
    """Função principal de processamento"""
    
    with status_output:
        clear_output()
        
        if not file_upload.value:
            print("❌ Por favor, selecione um arquivo CSV primeiro.")
            return
        
        try:
            print("🔄 Carregando arquivo...")
            
            # Carregar arquivo
            uploaded_file = list(file_upload.value.values())[0]
            content = uploaded_file['content']
            filename = uploaded_file['metadata']['name']
            
            # Ler CSV
            df = pd.read_csv(io.BytesIO(content))
            print(f"✅ Arquivo carregado: {filename} ({len(df)} registros)")
            
            # Validar formato
            print("🔍 Validando formato do arquivo...")
            is_valid, validation_message = validate_csv_format(df)
            
            if not is_valid:
                print(f"❌ Erro de formato: {validation_message}")
                print("\n📋 Formato esperado:")
                print("   - event_id (numérico)")
                print("   - plane (texto)")
                print("   - sensor (texto)")
                print("   - x_amplitude (numérico)")
                print("   - y_amplitude (numérico)")
                print("   - z_amplitude (numérico)")
                return
            
            print("✅ Formato válido!")
            
            # Processar análise
            print("⚡ Calculando energia total e detectando ruídos...")
            df_processed = calculate_energy_analysis(df)
            print("✅ Análise concluída!")
            
            # Gerar timestamp para nome do arquivo
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_filename = f"analise_sismica_resultados_{timestamp}.csv"
            
            print(f"📊 Gerando visualizações e relatório...")
            
        except Exception as e:
            print(f"❌ Erro durante o processamento: {str(e)}")
            return
    
    # Mostrar resultados
    with results_output:
        clear_output()
        
        try:
            # Estatísticas resumidas
            stats_html = generate_summary_stats(df_processed)
            display(HTML(stats_html))
            
            # Gráficos
            print("\n📈 Gráficos de Análise:")
            create_analysis_plots(df_processed)
            
            # Link de download
            download_html = create_download_link(df_processed, output_filename)
            display(HTML(download_html))
            
            # Mostrar primeiras linhas dos resultados
            print("\n📋 Primeiras linhas dos resultados processados:")
            display(df_processed[[
                'event_id', 'sensor', 'total_energy', 
                'event_energy_class', 'potential_noise_event', 'high_energy_flag'
            ]].head(10))
            
        except Exception as e:
            print(f"❌ Erro ao gerar visualizações: {str(e)}")
    
    with status_output:
        print("🎉 Análise completa! Verifique os resultados abaixo.")

# Conectar função ao botão
process_button.on_click(process_analysis)

print("✅ Interface interativa pronta!")

---

## 📖 Sobre a Análise

### 🔬 **Cálculos Realizados:**
- **Energia Total**: E = √(x² + y² + z²)
- **Classificação por Evento**: Baixa, Média, Alta energia
- **Detecção de Ruídos**: Baseada na variabilidade estatística
- **Ranking de Energia**: Posição relativa de cada registro

### 📊 **Critérios de Classificação:**
- **Eventos Limpos**: Baixa variabilidade interna
- **Possível Ruído**: Alta variabilidade (acima do percentil 90)
- **Alta Energia**: Registros acima do percentil 95

### 💾 **Dados de Saída:**
O arquivo CSV gerado contém todas as colunas originais mais:
- `total_energy` - Energia total calculada
- `energy_rank` - Ranking de energia
- `energy_percentile` - Percentil de energia
- `event_energy_class` - Classificação do evento
- `potential_noise_event` - Flag de possível ruído
- `high_energy_flag` - Flag de alta energia
- Estatísticas por evento: `event_mean`, `event_std`, `event_count`

---
*Desenvolvido para análise de dados sismológicos - Projeto Braskem*